## Column Transformer 

In [16]:
import numpy as np
import pandas as pd

In [17]:
df = pd.read_csv('covid_toy.csv')
df.head(3)

age gender  fever cough     city has_covid
0   60   Male  103.0  Mild  Kolkata        No
1   27   Male  100.0  Mild    Delhi       Yes
2   42   Male  101.0  Mild    Delhi        No

Now lets preprocess the data.    
* age - > SimpleImputer  
* gender and city  -> NominalEncoder (OneHotEncoder) 
* cough - > Ordinal Encoder

### Without column Transformer   

We have to manually apply different tekniks to different column   

In [18]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

Before processing lets train_test_split  

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train ,X_test, y_train ,y_test = train_test_split(df.drop(['has_covid'],axis = 1) , df['has_covid']
                                                   ,test_size = 0.2 , random_state = 42)

In [21]:
X_train.head(3)

age  gender  fever cough     city
55   81  Female  101.0  Mild   Mumbai
88    5  Female  100.0  Mild  Kolkata
26   19  Female  100.0  Mild  Kolkata

### Fever (SImpleImputer )

In [22]:
si = SimpleImputer()

In [23]:
X_train_fever =  si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [24]:
X_train

age  gender  fever   cough       city
55   81  Female  101.0    Mild     Mumbai
88    5  Female  100.0    Mild    Kolkata
26   19  Female  100.0    Mild    Kolkata
42   27    Male  100.0    Mild      Delhi
69   73  Female  103.0    Mild      Delhi
..  ...     ...    ...     ...        ...
60   24  Female  102.0  Strong  Bangalore
71   75  Female  104.0  Strong      Delhi
14   51    Male  104.0    Mild  Bangalore
92   82  Female  102.0  Strong    Kolkata
51   11  Female  100.0  Strong    Kolkata

[80 rows x 5 columns]

### OneHotEncoding

Applying on Gender and city

In [26]:
ohe = OneHotEncoder(drop='first',sparse = False )
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

E:\conda_project\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
E:\conda_project\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

### OrdinalEncoding

Ordinal Encoding is used when their is priority in dataset. 

In [33]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [34]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [35]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## NOW USING COLUMN  TRANSFORMER

In [36]:
from sklearn.compose import ColumnTransformer

In [48]:
transformer = ColumnTransformer(transformers = [
                                ('tnf1',SimpleImputer(),['fever']),
                                ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                                ('tnf3',OneHotEncoder(drop='first' , sparse = False ),['gender','city'])
                                ],
                                remainder ='passthrough')
                            

In [49]:
transformer.fit_transform(X_train).shape

E:\conda_project\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)